In [3]:
# Import libraries
import numpy as np
import MDAnalysis as mda
import nmrformd as nmrmd

# For figure plotting
import sys
from matplotlib import pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
fontsize = 30
font = {'family': 'sans', 'color':  'black', 'weight': 'normal', 'size': fontsize}
myblue = [0/ 255, 150/255, 177/ 255]
lightgray = [0.1, 0.1, 0.1]
darkgray = [0.9, 0.9, 0.9]
myred= [255/ 255, 77/255, 77/ 255]
sys.path.append('../../shared/')
from tools import figure_improvement, save_figure

# Path to data
datapath = "../../raw-data/bulk-water/N398/"

# Create a MDAnalysis universe
u = mda.Universe(datapath+"topology.data", datapath+"prod.xtc")
u.transfer_to_memory(stop=501) # to analyse only a subpart of the trajectory

In [4]:
# Print a few information
n_molecules = u.atoms.n_residues
print(f"The number of water molecules is {n_molecules}")
timestep = np.int32(u.trajectory.dt)
print(f"The timestep is {timestep} ps")
total_time = np.int32(u.trajectory.totaltime)
print(f"The total simulation time is {total_time} ps")

The number of water molecules is 398
The timestep is 1 ps
The total simulation time is 500 ps


In [ ]:
# Run NMRforMD
group_i = u.select_atoms("type 2")
nmr_result = nmrmd.NMR(u, group_i, number_i=80) # 3-4 minutes to complete

In [ ]:
# T1 bulk water - tutorial figure
f_1 = 2600
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.f[:-f_1], 1/nmr_result.R1[:-f_1], 'o', markersize=8, color=myblue)
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$T_1$ (s)',
                       xlim=(1e2, 2e5), ylim=(1, 100))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/T1', save=True)

In [ ]:
# Correlation function water - tutorial figure
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.t, nmr_result.gij[0]/nmr_result.gij[0][0], 'o', markersize=8, color=myblue)
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)} (t) / G^{(0)} (0)$',
                       xlim=(1, 200), ylim=(0.0001, 1))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/Gij', save=True)

In [ ]:
# Separate intra and inter motion
nmr_result_intra = nmrmd.NMR(u, group_i, type_analysis="intra_molecular", number_i=120) # 2-3 minutes
nmr_result_inter = nmrmd.NMR(u, group_i, type_analysis="inter_molecular", number_i=80) # 3-4 minutes

In [ ]:
f_1 = 2600
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.f[:-f_1], nmr_result.R1[:-f_1], 'o', markersize=8, color=mygray, label="Total")
    ax1.loglog(nmr_result_intra.f[:-f_1], nmr_result_intra.R1[:-f_1], '^', markersize=8, color=myred, label="Intra-molecular")
    ax1.loglog(nmr_result_inter.f[:-f_1], nmr_result_inter.R1[:-f_1], 's', markersize=8, color=myblue, label="Inter-molecular")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$R_1$ (s$^{-1}$)',
                       xlim=(1e2, 2e5), ylim=(0.01, 1))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/R1-intra-inter', save=True)

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    #ax1.loglog(nmr_result.t, nmr_result.gij[0]/nmr_result.gij[0][0], 'o', markersize=8, color=myblue, label="Total")
    ax1.loglog(nmr_result_intra.t, nmr_result_intra.gij[0]/nmr_result.gij[0][0], '^', markersize=8, color=myred, label="Intra-molecular")
    ax1.loglog(nmr_result_inter.t, nmr_result_inter.gij[0]/nmr_result.gij[0][0], 's', markersize=8, color=myblue, label="Inter-molecular")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)} (t) / G^{(0)} (0)$',
                       xlim=(1, 500), ylim=(0.00001, 1))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/Gij-intra-inter', save=True)